In [54]:
%pip install requests mido python-rtmidi

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import requests

def fetch_air_quality_data(latitude, longitude, api_token):
    # Prepare the endpoint and parameters
    endpoint = f"https://api.waqi.info/feed/geo:{latitude};{longitude}/"
    params = {
        "token": api_token
    }

    # Make the request
    response = requests.get(endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None  # or handle errors as appropriate

# Replace with your actual values
api_token = "42ed33d5b7c9a020591a73b08b51a8a0d299ed6d"
latitude = 40.729513
longitude = -73.997460 #NYU Steinhardt location

# Fetch and print data
air_quality_data = fetch_air_quality_data(latitude, longitude, api_token)
print(air_quality_data)


{'status': 'ok', 'data': {'aqi': 24, 'idx': 3307, 'attributions': [{'url': 'http://www.dec.ny.gov/', 'name': 'New York State Department of Environmental Conservation (NYSDEC)', 'logo': 'US-NYDEC.png'}, {'url': 'http://www.airnow.gov/', 'name': 'Air Now - US EPA'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [40.7143528, -74.0059731], 'name': 'New York, USA', 'url': 'https://aqicn.org/city/usa/newyork', 'location': ''}, 'dominentpol': 'o3', 'iaqi': {'co': {'v': 2.3}, 'h': {'v': 45.9}, 'no2': {'v': 3.8}, 'o3': {'v': 24.1}, 'p': {'v': 1005}, 'pm25': {'v': 20}, 't': {'v': 15.1}, 'w': {'v': 9.2}}, 'time': {'s': '2023-10-22 13:00:00', 'tz': '-04:00', 'v': 1697979600, 'iso': '2023-10-22T13:00:00-04:00'}, 'forecast': {'daily': {'o3': [{'avg': 15, 'day': '2023-10-20', 'max': 18, 'min': 10}, {'avg': 4, 'day': '2023-10-21', 'max': 11, 'min': 1}, {'avg': 10, 'day': '2023-10-22', 'max': 16, 'min': 6}, {'avg': 7, 'day': '2023-10-23', 'max': 15, 'min': 1

In [63]:
def parse_air_quality_data(data):
    if data and data.get('status') == 'ok':
        # Extracting the air quality index (AQI)
        aqi = data['data']['aqi']

        # Extracting individual pollutant data
        iaqi = data['data']['iaqi']

        #access individual pollutants depending on availability
        pm25 = iaqi.get('pm25', {}).get('v')  # PM2.5 data
        pm10 = iaqi.get('pm10', {}).get('v')  # PM10 data
        co = iaqi.get('co', {}).get('v')  # Carbon Monoxide data
        no2 = iaqi.get('no2', {}).get('v')  # Nitrogen Dioxide data
        o3 = iaqi.get('o3', {}).get('v')  # Ozone data


        # Returning a dictionary with the parsed data
        return {
            'aqi': aqi,
            'pm25': pm25,
            'pm10': pm10,
            'co': co,
            'no2': no2,
            'o3': o3
            # Add more pollutants here
        }
    else:
        print("Error: Unable to retrieve or parse data")
        return None

# Assuming air_quality_data contains the response from the API
parsed_data = parse_air_quality_data(air_quality_data)
print(parsed_data)


{'aqi': 24, 'pm25': 20, 'pm10': None, 'co': 2.3, 'no2': 3.8, 'o3': 24.1}


In [62]:
print(pm25_midi, co_midi, no2_midi, o3_midi)

3 5 3 8


In [58]:
import mido
import time

# Function to normalize data
def normalize_data(value, min_value, max_value, min_midi, max_midi):
    return ((value - min_value) / (max_value - min_value)) * (max_midi - min_midi) + min_midi

In [59]:
import mido

# List all available MIDI output ports
print(mido.get_output_names())

['Microsoft GS Wavetable Synth 0', 'MyVirtualMIDI 1', 'MyVirtualMIDI 1 2']


In [5]:
# Function to send MIDI messages
def send_midi_control_change(control, value, midi_port):
    # control is the control number (e.g., 1 for modulation wheel, 7 for volume, etc.)
    # value is the control value (0-127)
    msg = mido.Message('control_change', control=control, value=value)
    midi_port.send(msg)


In [120]:
# Testing
import mido
import time

# Open the MIDI port
outport = mido.open_output('MyVirtualMIDI 1')

# Send a note on message
note_on = mido.Message('note_on', note=60, velocity=64)
outport.send(note_on)

# Wait for a second
time.sleep(1)

# Send a note off message
note_off = mido.Message('note_off', note=60, velocity=64)
outport.send(note_off)

# Close the MIDI port
outport.close()

print("MIDI messages sent.")


MIDI messages sent.


In [121]:
import mido
import time

# Function to map PM2.5 level to a MIDI note
def pm25_to_midi(pm25_level):
    # Ensure pm25_level is within the expected range
    pm25_level = max(min(pm25_level, 500), 0)
    # Invert the mapping: higher PM2.5 levels should produce higher notes
    midi_note = 108 - int((pm25_level / 500) * (108 - 21))
    midi_note = max(min(midi_note, 108), 21)  # Ensure midi_note is in the range 21-108
    return midi_note

# Function to map PM2.5 level to a MIDI velocity
def pm25_to_midi(pm25_level):
    # Ensure pm25_level is within the expected range
    pm25_level = max(min(pm25_level, 500), 0)

    # Define the range of MIDI notes you want to use
    lowest_note = 40  # This is E2 on a piano
    highest_note = 88  # This is E6 on a piano

    # Map the PM2.5 level to a MIDI note within your defined range
    midi_note = int((pm25_level / 500) * (highest_note - lowest_note) + lowest_note)

    # Ensure midi_note is within your defined range
    midi_note = max(min(midi_note, highest_note), lowest_note)

    return midi_note


# Create a virtual MIDI port
with mido.open_output('MyVirtualMIDI 1') as outport:
    # Assuming air_quality_data contains the response from the API
    parsed_data = parse_air_quality_data(air_quality_data)

    # Get the PM2.5 level, if available
    pm25_level = parsed_data.get('pm25', 0)  # Default to 0 if 'pm25' is not available

    # Convert the PM2.5 level to a MIDI note
    note = pm25_to_midi(pm25_level)

    # Convert the PM2.5 level to a MIDI velocity
    velocity = pm25_to_velocity(pm25_level)

    # Send a "note on" message to play the note
    note_on = mido.Message('note_on', note=note, velocity=velocity)
    outport.send(note_on)

    # Wait for a bit to let the note play
    time.sleep(1)

    # Send a "note off" message to stop the note
    note_off = mido.Message('note_off', note=note, velocity=velocity)
    outport.send(note_off)

    # Sleep for a bit to ensure messages are sent before closing the port
    time.sleep(0.1)

print("MIDI messages sent.")


MIDI messages sent.


In [124]:
import mido
import time
import random  # for randomizing parameters

# Function to map PM2.5 level to a MIDI note
def pm25_to_midi(pm25_level):
    # Ensure pm25_level is within the expected range
    pm25_level = max(min(pm25_level, 500), 0)
    # Invert the mapping: higher PM2.5 levels should produce higher notes
    midi_note = 108 - int((pm25_level / 500) * (108 - 21))
    midi_note = max(min(midi_note, 108), 21)  # Ensure midi_note is in the range 21-108
    return midi_note

# New function to map PM2.5 level to a MIDI velocity
def pm25_to_velocity(pm25_level):
    # Ensure pm25_level is within the expected range
    pm25_level = max(min(pm25_level, 500), 0)
    # Map the PM2.5 level to a MIDI velocity
    midi_velocity = int((pm25_level / 500) * 127)
    return midi_velocity

# Create a virtual MIDI port
with mido.open_output('MyVirtualMIDI 1') as outport:
    # Assuming air_quality_data contains the response from the API
    parsed_data = parse_air_quality_data(air_quality_data)

    # Get the PM2.5 level, if available
    pm25_level = parsed_data.get('pm25', 0)  # Default to 0 if 'pm25' is not available

    # Convert the PM2.5 level to a MIDI note
    note = pm25_to_midi(pm25_level)

    # Convert the PM2.5 level to a MIDI velocity
    velocity = pm25_to_velocity(pm25_level)

    for _ in range(5):  # Play 5 notes
        # Randomize some other parameter, for example, the instrument
        instrument = random.choice(range(1, 128))  # MIDI instruments are numbered 1-128
        outport.send(mido.Message('program_change', program=instrument))

        # Randomize pitch bend
        pitch_bend = random.choice(range(-8192, 8191))
        outport.send(mido.Message('pitchwheel', pitch=pitch_bend))

        # Send a "note on" message to play the note
        note_on = mido.Message('note_on', note=note, velocity=velocity)
        outport.send(note_on)

        # Wait for a bit to let the note play
        time.sleep(1)

        # Send a "note off" message to stop the note
        note_off = mido.Message('note_off', note=note, velocity=velocity)
        outport.send(note_off)

        # Sleep for a bit to ensure messages are sent before closing the port
        time.sleep(0.1)

print("MIDI messages sent.")


MIDI messages sent.
